In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/UCD/ECS-289G-DL
import sys
sys.path.append('/content/drive/MyDrive/UCD/ECS-289G-DL')

/content/drive/MyDrive/UCD/ECS-289G-DL


In [ ]:
!pip install pytorch_lightning
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
from auto_encoder import AutoEncoder
from masked_dataset import CIFAR10DataModule, IMDBDataModule
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
from pytorch_lightning.loggers import TensorBoardLogger


In [ ]:
cifar10 = CIFAR10DataModule()

In [ ]:
in_channel_size = 3
num_epochs = 10
model = AutoEncoder(in_channel_size=3)

trainer = pl.Trainer(
    default_root_dir=os.getcwd(),
    logger=TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs"),
    auto_lr_find=True, 
    auto_scale_batch_size=True,
    accelerator="auto",
    log_every_n_steps=1, 
    max_epochs=50, 
    min_epochs=10,
    callbacks=[
        ModelCheckpoint(save_last=True),
        LearningRateMonitor("epoch"),
        EarlyStopping(monitor="val_loss", mode="min", patience=5)
    ])
trainer.logger._log_graph = True

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.tune(model, cifar10)

  0%|          | 0/2 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=3` reached.
INFO:pytorch_lightning.utilities.rank_zero:Batch size 2 succeeded, trying batch size 4
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=3` reached.
INFO:pytorch_lightning.utilities.rank_zero:Batch size 4 succeeded, trying batch size 8
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=3` reached.
INFO:pytorch_lightni

  0%|          | 0/2 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.0009120108393559097
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/drive/MyDrive/UCD/ECS-289G-DL/.lr_find_0e70e086-1f30-4f2f-83bc-5a40e65316c2.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/drive/MyDrive/UCD/ECS-289G-DL/.lr_find_0e70e086-1f30-4f2f-83bc-5a40e65316c2.ckpt


{'scale_batch_size': 16,
 'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7f91832005d0>}

In [ ]:
trainer.fit(model, cifar10)

  0%|          | 0/2 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type               | Params
-----------------------------------------------
0 | e1      | EncoderBlock       | 13.2 K
1 | e2      | EncoderBlock       | 74.5 K
2 | e3      | EncoderBlock       | 296 K 
3 | e4      | EncoderBlock       | 1.2 M 
4 | b       | ConvolutionalBlock | 4.7 M 
5 | d1      | DecoderBlock       | 3.4 M 
6 | d2      | DecoderBlock       | 853 K 
7 | d3      | DecoderBlock       | 213 K 
8 | d4      | DecoderBlock       | 53.7 K
9 | outputs | Conv1d             | 195   
-----------------------------------------------
10.8 M    Trainable params
0         Non-trainable params
10.8 M    Total params
43.299    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/tensorboard.py:249: UserWarning: Could not log computational graph since the `model.example_input_array` attribute i

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]